In [1]:
import pandas as pd
data = pd.read_csv('traindata2')

In [2]:
alltext = []
for i in data['text']:
    tl = i.split(' ')
    alltext.append(tl)

In [3]:
from gensim.corpora import Dictionary
dct = Dictionary(alltext)

In [4]:
from keras.preprocessing import sequence
X = [dct.doc2idx(txt) for txt in alltext]
X =sequence.pad_sequences(X, padding='post', value=len(dct))

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,data.iloc[:,-1],test_size=0.3,random_state=0)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras_self_attention import SeqSelfAttention

In [7]:
model = Sequential()
model.add(Embedding(len(dct)+1,128,input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(SeqSelfAttention())
model.add(LSTM(64))
model.add(Dense(4,activation='softmax'))
model.summary()
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1337, 128)         3421184   
                                                                 
 bidirectional (Bidirectiona  (None, 1337, 128)        98816     
 l)                                                              
                                                                 
 seq_self_attention (SeqSelf  (None, 1337, 128)        8257      
 Attention)                                                      
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 4)                 260       
                                                                 
Total params: 3,577,925
Trainable params: 3,577,925
Non-

In [8]:
model.fit(x=X_train,y=y_train, validation_split=0.1, epochs=10, verbose=1)

Epoch 1/10
9/9 [==============================] - 176s 19s/step - loss: 1.3855 - accuracy: 0.2644 - val_loss: 1.3814 - val_accuracy: 0.1667
Epoch 2/10
9/9 [==============================] - 165s 18s/step - loss: 1.3744 - accuracy: 0.3372 - val_loss: 1.3637 - val_accuracy: 0.4000
Epoch 3/10
9/9 [==============================] - 183s 20s/step - loss: 1.3727 - accuracy: 0.2797 - val_loss: 1.3642 - val_accuracy: 0.4333
Epoch 4/10
9/9 [==============================] - 191s 21s/step - loss: 1.3668 - accuracy: 0.3525 - val_loss: 1.3913 - val_accuracy: 0.1333
Epoch 5/10
9/9 [==============================] - 167s 18s/step - loss: 1.3468 - accuracy: 0.4138 - val_loss: 1.3806 - val_accuracy: 0.3333
Epoch 6/10
9/9 [==============================] - 160s 18s/step - loss: 1.1414 - accuracy: 0.6092 - val_loss: 1.5165 - val_accuracy: 0.2667
Epoch 7/10
9/9 [==============================] - 163s 18s/step - loss: 0.7802 - accuracy: 0.6935 - val_loss: 1.7701 - val_accuracy: 0.2333
Epoch 8/10
9/9 [====

In [9]:
model.evaluate(X_test, y_test ,batch_size=10)

13/13 [==============================] - 22s 2s/step - loss: 2.2530 - accuracy: 0.2640


[2.253028392791748, 0.2639999985694885]

In [10]:
model.predict(X_test)

array([[0.8179723 , 0.02427649, 0.0587024 , 0.09904873],
       [0.08236648, 0.10611476, 0.02177276, 0.78974605],
       [0.01155893, 0.597587  , 0.37728888, 0.01356519],
       [0.05633209, 0.10534123, 0.01437872, 0.823948  ],
       [0.06393662, 0.06456573, 0.01162459, 0.859873  ],
       [0.04076198, 0.47486892, 0.03608721, 0.44828188],
       [0.73327786, 0.03522853, 0.08441232, 0.14708133],
       [0.10967056, 0.05776304, 0.01696835, 0.815598  ],
       [0.6144182 , 0.03373393, 0.28531438, 0.06653347],
       [0.03978772, 0.6728011 , 0.09771877, 0.18969251],
       [0.07074068, 0.11601183, 0.02043493, 0.7928126 ],
       [0.22258286, 0.08367746, 0.04552   , 0.64821965],
       [0.14733218, 0.08206933, 0.02753754, 0.74306095],
       [0.00407361, 0.9557882 , 0.01730923, 0.02282896],
       [0.8218292 , 0.01993794, 0.09666279, 0.06157006],
       [0.11182579, 0.06032493, 0.01823143, 0.8096179 ],
       [0.02395608, 0.7108268 , 0.22380371, 0.04141334],
       [0.03589859, 0.07212361,

In [ ]:
from keras.models import load_model
model.save('kerassave')
newmodel = load_model('kerassave')